In [43]:
import pandas as pd
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

PassengerId- 각 승객에 대한 고유한 ID. 각 ID는 승객이 함께 여행하는 그룹을 나타내는 형식을 취하며, gggg_pp그룹 내의 번호입니다. 그룹에 속한 사람들은 종종 가족이지만 항상 그런 것은 아닙니다.ggggpp
HomePlanet- 승객이 출발한 행성, 일반적으로 영주권이 있는 행성입니다.
CryoSleep- 승객이 항해 기간 동안 정지된 애니메이션 상태에 들어가도록 선택했는지 여부를 나타냅니다. 냉동 수면 상태의 승객은 객실에 갇힙니다.
Cabin- 승객이 머물고 있는 객실 번호. 형식을 취하며 deck/num/side, 여기서는 Port 또는 Starboard 중 하나 를side 사용할 수 있습니다 .PS
Destination- 승객이 하선할 행성.
Age- 승객의 나이.
VIP- 승객이 항해 중 특별 VIP 서비스에 대한 비용을 지불했는지 여부.
RoomService, FoodCourt, ShoppingMall, Spa, VRDeck- 승객이 타이타닉호의 다양한 고급 편의시설에 대해 청구한 금액입니다.
Name- 승객의 성과 이름.
Transported- 승객이 다른 차원으로 이동했는지 여부. 이것이 타겟이고, 당신이 예측하려는 열입니다.

In [44]:
FILE1 = '../data/spaceship-titanic/train.csv'
FILE2 = '../data/spaceship-titanic/test.csv'

traindf=pd.read_csv(FILE1)
testdf=pd.read_csv(FILE2)
featuredf=traindf.drop(['Transported', 'Name', 'Cabin', 'PassengerId'],axis=1)
target = traindf['Transported']
featuredf
target=pd.DataFrame(target)

In [45]:
# One-hot encoding 처리할 컬럼 선택
X_features_ohe = pd.get_dummies(featuredf[['HomePlanet', 'CryoSleep', 'Destination', 'VIP']], drop_first=True)

# True/False 값을 1/0으로 변환
X_features_ohe = X_features_ohe.applymap(lambda x: 1 if x is True else (0 if x is False else x))

# 결과 확인
X_features_ohe

,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True
0,1,0,0,0,1,0
1,0,0,0,0,1,0
2,1,0,0,0,1,1
3,1,0,0,0,1,0
4,0,0,0,0,1,0
...,...,...,...,...,...,...
8688,1,0,0,0,0,1
8689,0,0,1,1,0,0
8690,0,0,0,0,1,0
8691,1,0,0,0,0,0


In [46]:
# featuredf와 X_features_ohe를 열 기준으로 결합
combined_df = pd.concat([featuredf.drop(['HomePlanet','CryoSleep','Destination','VIP'], axis=1), X_features_ohe], axis=1)

# 결합된 데이터프레임 확인
combined_df.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True
0,39.0,0.0,0.0,0.0,0.0,0.0,1,0,0,0,1,0
1,24.0,109.0,9.0,25.0,549.0,44.0,0,0,0,0,1,0
2,58.0,43.0,3576.0,0.0,6715.0,49.0,1,0,0,0,1,1
3,33.0,0.0,1283.0,371.0,3329.0,193.0,1,0,0,0,1,0
4,16.0,303.0,70.0,151.0,565.0,2.0,0,0,0,0,1,0


In [48]:
target_ohe = pd.get_dummies(target, columns=target.columns,drop_first=True)
target_ohe
target_ohe = target_ohe.applymap(lambda x: 1 if x is True else (0 if x is False else x))
target_ohe

,Transported_True
0,0
1,1
2,0
3,0
4,1
...,...
8688,0
8689,0
8690,1
8691,0


In [50]:
# 전부 중복인 값, 한개라도 중복인값 체크
# 대부분 각 컬럼별 결측치, 삭제아닌 최대한 채우기로 결정
combined_df.isna().all(axis=1).sum(), combined_df.isna().any(axis=1).sum()

(0, 1073)

In [52]:
# 모든 금액이 결측치인 데이터 체크 없으므로 개인별 총 지불한 금액 평균치를 대입하기로 결정
space_cols =combined_df.columns[[7,8,9,10,11]].to_list()
combined_df[space_cols].isna().all(axis=1).sum()

0